In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import sympy as sp
import math
# !pip3 install git+https://github.com/Fangyh09/pytorch-receptive-field.git
from torch_receptive_field import receptive_field
from torchinfo import summary

import models as cifar_models

In [3]:
model = cifar_models.NetBN()

counter = 0
# Loop through each layer of the model
c_in, h_in, w_in = 3, 32, 32
for name, layer in model.named_children():
    print(f"- Layer details: {name} | k={layer.kernel_size} | s={layer.stride} | p={layer.padding}")
    if isinstance(layer, nn.Conv2d):
        out_channels = layer.out_channels
        k, s, p = layer.kernel_size[0], layer.stride[0], layer.padding[0]
        c_o, h_o, w_o = out_channels, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
        print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}")
        c_in, h_in, w_in = c_o, h_o, w_o
    
    elif isinstance(layer, nn.MaxPool2d):
        k, s, p = layer.kernel_size, layer.stride, layer.padding
        c_o, h_o, w_o = c_in, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
        print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}\n")
        c_in, h_in, w_in = c_o, h_o, w_o

    elif isinstance(layer, nn.AvgPool2d):
        k, s, p = layer.kernel_size, layer.stride, layer.padding
        c_o, h_o, w_o = c_in, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
        print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}\n")
        c_in, h_in, w_in = c_o, h_o, w_o

    else:
        print('layer not implemented')
        break


- Layer details: conv1 | k=(3, 3) | s=(1, 1) | p=(1, 1)
-------------------------------- no. of channels: 32 | feature map dim: 32x32
- Layer details: conv2 | k=(3, 3) | s=(1, 1) | p=(1, 1)
-------------------------------- no. of channels: 64 | feature map dim: 32x32
- Layer details: conv3 | k=(1, 1) | s=(1, 1) | p=(0, 0)
-------------------------------- no. of channels: 64 | feature map dim: 32x32
- Layer details: pool1 | k=2 | s=2 | p=0
-------------------------------- no. of channels: 64 | feature map dim: 16x16

- Layer details: conv4 | k=(3, 3) | s=(1, 1) | p=(1, 1)
-------------------------------- no. of channels: 128 | feature map dim: 16x16
- Layer details: conv5 | k=(3, 3) | s=(1, 1) | p=(1, 1)
-------------------------------- no. of channels: 256 | feature map dim: 16x16
- Layer details: conv6 | k=(3, 3) | s=(1, 1) | p=(1, 1)
-------------------------------- no. of channels: 512 | feature map dim: 16x16
- Layer details: conv7 | k=(1, 1) | s=(1, 1) | p=(0, 0)
-----------------

In [4]:
# receptive_field(model, input_size=(channels, H, W))
receptive_field(model, input_size=(3, 32, 32))

------------------------------------------------------------------------------
        Layer (type)    map size      start       jump receptive_field 
        0               [32, 32]        0.5        1.0             1.0 
        1               [32, 32]        0.5        1.0             3.0 
        2               [32, 32]        0.5        1.0             5.0 
        3               [32, 32]        0.5        1.0             5.0 
        4               [16, 16]        1.0        2.0             6.0 
        5               [16, 16]        1.0        2.0            10.0 
        6               [16, 16]        1.0        2.0            14.0 
        7               [16, 16]        1.0        2.0            18.0 
        8               [16, 16]        1.0        2.0            18.0 
        9                 [8, 8]        2.0        4.0            20.0 
        10                [8, 8]        2.0        4.0            28.0 
        11                [8, 8]        2.0        4.0   

OrderedDict([('0',
              OrderedDict([('j', 1.0),
                           ('r', 1.0),
                           ('start', 0.5),
                           ('conv_stage', True),
                           ('output_shape', [-1, 3, 32, 32])])),
             ('1',
              OrderedDict([('j', 1.0),
                           ('r', 3.0),
                           ('start', 0.5),
                           ('input_shape', [-1, 3, 32, 32]),
                           ('output_shape', [-1, 32, 32, 32])])),
             ('2',
              OrderedDict([('j', 1.0),
                           ('r', 5.0),
                           ('start', 0.5),
                           ('input_shape', [-1, 32, 32, 32]),
                           ('output_shape', [-1, 64, 32, 32])])),
             ('3',
              OrderedDict([('j', 1.0),
                           ('r', 5.0),
                           ('start', 0.5),
                           ('input_shape', [-1, 64, 32, 32]),
         

In [5]:
model.summary(input_size=(1, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
NetBN                                    [1, 10]                   --
├─Conv2d: 1-1                            [1, 32, 32, 32]           896
├─Conv2d: 1-2                            [1, 64, 32, 32]           18,496
├─Conv2d: 1-3                            [1, 64, 32, 32]           4,160
├─MaxPool2d: 1-4                         [1, 64, 16, 16]           --
├─Conv2d: 1-5                            [1, 128, 16, 16]          73,856
├─Conv2d: 1-6                            [1, 256, 16, 16]          295,168
├─Conv2d: 1-7                            [1, 512, 16, 16]          1,180,160
├─Conv2d: 1-8                            [1, 512, 16, 16]          262,656
├─MaxPool2d: 1-9                         [1, 512, 8, 8]            --
├─Conv2d: 1-10                           [1, 1024, 8, 8]           4,719,616
├─Conv2d: 1-11                           [1, 2048, 8, 8]           18,876,416
├─Conv2d: 1-12                           